In [14]:
from models import * 
import import_ipynb
from data_set_loader import *
import torch.optim
from training import * 
from linear_eval import *

## Pre-train SimCLR for a few epochs

Crops, Flips, and Normalization augmentations

In [2]:
# Increase batch size from 10 to 20 to make it run faster. 

batch_size = 20

train_data = ContrastiveCIFAR10()
val_data = ContrastiveCIFAR10(mode = 'val')
train_dl = DataLoader(train_data, batch_size)
val_dl = DataLoader(val_data, batch_size)

In [3]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [4]:
from pytorch_metric_learning.losses import NTXentLoss

In [6]:
pretrain = SimCLR().to(device)
criterion = NTXentLoss(temperature=0.1).to(device)
optimizer = torch.optim.Adam(pretrain.parameters())

model_train(train_dl, val_dl, pretrain, criterion, optimizer, device=device)

/home/sshad/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/sshad/anaconda3/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch: 1/10 	 T Loss: 1.5501966186463834          	 V Loss: 0.339189132720232
Epoch: 2/10 	 T Loss: 0.7132333189323544          	 V Loss: 0.2085491861253977
Epoch: 3/10 	 T Loss: 0.5508875146433712          	 V Loss: 0.19696026097238065


KeyboardInterrupt: 

## Evaluate on Test Data

Currently uses test data bc of the way the dataset is written, should be able to change it. 

In [30]:
feature_model = LinearHead(pretrain.resnet).to(device)
optimizer = torch.optim.SGD(feature_model.fc.parameters()) #default settings

In [31]:
test_data = ContrastiveCIFAR10(mode = 'test')
test_dl = DataLoader(test_data, batch_size)

In [71]:
def evaluate_model(train_loader, test_loader, model, optimizer, epochs=10, device='cpu'): 
    
    model = model.to(device)
    for epoch in range(epochs): 
        
        train_loss = 0
        train_size = 0

        for image, y in test_loader: 
            
            image = image.to(device)
            y = y.to(device).long()


            out = model(image)
            loss = F.cross_entropy(out, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()*y.size(0)
            train_size += y.size(0)

        with torch.no_grad(): 
            
            total = 0
            correct = 0
            
            for image, y in test_loader: 
                image = image.to(device)
                y = y.to(device)

                out = model(image)
                preds = torch.argmax(out, dim=1)
                total += y.size(0)
                correct += (y==preds).sum().item()

        print(f"Epoch {epoch+1}/{epochs} Train Loss = {train_loss/train_size} \
        Test Accuracy = {correct/total}")

In [72]:
evaluate_model(train_dl, test_dl, model=feature_model, optimizer=optimizer)

Epoch 1/10 Train Loss = 3.3202277450561524         Test Accuracy = 0.1761
Epoch 2/10 Train Loss = 3.2999357199668884         Test Accuracy = 0.1822
Epoch 3/10 Train Loss = 3.2971250438690185         Test Accuracy = 0.1876


KeyboardInterrupt: 